[View in Colaboratory](https://colab.research.google.com/github/Hoiy/kaggle-home-credit-default-risk/blob/master/model_lightgbm.ipynb)

In [0]:
import pandas as pd
import numpy as np
import dotenv
import os
from scipy import sparse
from sklearn.metrics import roc_auc_score
import pickle
import gc
import lightgbm as lgb

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

dotenv.load_dotenv('.env')

True

In [0]:
!mkdir prep
!gsutil rsync gs://{os.environ['GCP_BUCKET']}/prep prep

mkdir: prep: File exists
Building synchronization state...
Starting synchronization...


In [0]:
train = pd.read_parquet('./prep/application_train.snappy.parquet')
train.shape

test = pd.read_parquet('./prep/application_test.snappy.parquet')
test.shape

(48744, 1630)

In [0]:
y_train = train.TARGET
X_train = train.drop(columns=['TARGET'])
X_test = test

del train
del test
import gc
gc.collect()

from tqdm import tqdm

cat_cols = []
cont_cols = []
for col, dtype in tqdm(X_train.dtypes.iteritems()):
  if dtype == 'object':
    dtypes = pd.concat([X_train[col], X_test[col]]).astype('category').dtypes
    X_train[col] = X_train[col].astype(dtypes).cat.codes.astype(int)
    X_test[col] = X_test[col].astype(dtypes).cat.codes.astype(int)
    cat_cols.append(col)
  else:
    cont_cols.append(col)

1630it [01:56, 14.04it/s]


In [0]:
X_train.to_feather('X_train.feather')
y_train.to_frame().to_feather('y_train.feather')
X_test.to_feather('X_test.feather')
import pickle
with open('cols_type.pkl', 'wb') as f:
  pickle.dump([cat_cols, cont_cols], f)

In [0]:
X_train = pd.read_feather('X_train.feather')
y_train = pd.read_feather('y_train.feather')['TARGET']
X_test = pd.read_feather('X_test.feather')
import pickle
with open('cols_type.pkl', 'rb') as f:
  cat_cols, cont_cols = pickle.load(f)

In [0]:
K_FOLD=5
TEST_SIZE = 1/K_FOLD
RANDOM_SEED=42

#TODO: how to setup gpu lightgbm in colab
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
#     'min_data_in_leaf': 1000,
#     'num_leaves': 250,
#     'feature_fraction': 0.8,
#     'bagging_freq': 10,
#     'bagging_fraction': 0.5,
#     'max_depth': 8,
#     'min_data_in_leaf': 1000,
#     'device': 'gpu',
#     'num_leaves': 270,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.75,
    'bagging_freq': 2,
    'learning_rate': 0.0175,
#     'n_estimators': 10000,
#     'learning_rate': 0.02,
#     'num_leaves': 34,
#     'colsample_bytree': 0.9497036,
#     'subsample': 0.8715623,
#     'max_depth': 8,
#     'reg_alpha': 0.041545473,
#     'reg_lambda': 0.0735294,
#     'min_split_gain': 0.0222415,
#     'min_child_weight': 39.3259775,
    'verbose': 1
}

lgb_params = {
    'verbose_eval': 10,
}

In [0]:
ldata = lgb.Dataset(X_train, y_train, feature_name=X_train.columns.tolist())

results = lgb.cv(
    params,
    ldata,
    stratified=False,
    nfold=K_FOLD,
    seed=RANDOM_SEED,
    num_boost_round=10000,
    early_stopping_rounds=100,
    **lgb_params
)

KeyboardInterrupt: ignored

In [0]:
cv = results['auc-mean'][-1]
rounds = len(results['auc-mean'])
rounds, cv, K_FOLD, 1/K_FOLD

NameError: ignored

In [0]:
def fit_predict(X_train, y_train, X_test, n_folds=10, random_state=42):
    from sklearn.model_selection import KFold
    from sklearn.metrics import mean_squared_error, roc_auc_score

    sub_preds = np.zeros(X_test.shape[0])
    oof_preds = np.zeros(X_train.shape[0])
    
    folds = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    
    models = []
    for trn_idx, val_idx in folds.split(X_train):
        dtrain = lgb.Dataset(X_train.iloc[trn_idx], y_train.iloc[trn_idx], free_raw_data=False, categorical_feature=cat_cols)
        dval = lgb.Dataset(X_train.iloc[val_idx], y_train.iloc[val_idx], free_raw_data=False, categorical_feature=cat_cols)
      
        clf = lgb.train(
            params=params,
            train_set=dtrain,
            valid_sets=dval,
            num_boost_round=10000, 
            early_stopping_rounds=100,
            **lgb_params
        )
        models.append(clf)
        # Predict Out Of Fold and Test targets
        # Using lgb.train, predict will automatically select the best round for prediction
        
        oof_preds[val_idx] = clf.predict(X_train.iloc[val_idx])
        sub_preds += clf.predict(X_test) / folds.n_splits
        
        # Display current fold score
        print(roc_auc_score(y_train.iloc[val_idx], oof_preds[val_idx]))

    
    print('Full Out-Of-Fold score : %9.6f' % (roc_auc_score(y_train, oof_preds)))

    return oof_preds, sub_preds, models

In [0]:
oof_preds, sub_preds, models = fit_predict(X_train, y_train, X_test, n_folds=5)

/usr/local/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.735489
[20]	valid_0's auc: 0.739327
[30]	valid_0's auc: 0.740825
[40]	valid_0's auc: 0.742046
[50]	valid_0's auc: 0.743104
[60]	valid_0's auc: 0.744078
[70]	valid_0's auc: 0.744911
[80]	valid_0's auc: 0.745575
[90]	valid_0's auc: 0.746579
[100]	valid_0's auc: 0.747718
[110]	valid_0's auc: 0.749349
[120]	valid_0's auc: 0.750862
[130]	valid_0's auc: 0.752569
[140]	valid_0's auc: 0.754125
[150]	valid_0's auc: 0.75614
[160]	valid_0's auc: 0.758099
[170]	valid_0's auc: 0.759883
[180]	valid_0's auc: 0.761469
[190]	valid_0's auc: 0.763239
[200]	valid_0's auc: 0.764808
[210]	valid_0's auc: 0.766568
[220]	valid_0's auc: 0.768404
[230]	valid_0's auc: 0.770058
[240]	valid_0's auc: 0.771358
[250]	valid_0's auc: 0.772625
[260]	valid_0's auc: 0.77377
[270]	valid_0's auc: 0.774846
[280]	valid_0's auc: 0.775757
[290]	valid_0's auc: 0.776698
[300]	valid_0's auc: 0.777671
[310]	valid_0's auc: 0.778399
[320]	valid_0's au

In [0]:
lgb.plot_importance(models[0], figsize=(10, 100))

In [0]:
submission = X_test[['SK_ID_CURR']]
submission['TARGET'] = sub_preds
submission = submission.set_index('SK_ID_CURR')

SUBMISSION_FILE = 'lightgbm.csv'
SUBMISSION_MESSAGE='"LightGBM ALL %.4f"'%(roc_auc_score(y_train, oof_preds))

submission.to_csv(SUBMISSION_FILE)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
!kaggle competitions submit -f '{SUBMISSION_FILE}' -m '{SUBMISSION_MESSAGE}'

Using competition: home-credit-default-risk
Successfully submitted to Home Credit Default Risk